In [1]:
import pandas as pd
import numpy as np
import pymysql
from sqlalchemy import create_engine
import re
import os
from functools import reduce
pd.options.mode.chained_assignment = None

In [2]:
import math
import matplotlib.pyplot as plt
import datetime as dt
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Dropout, TimeDistributed
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard

from tensorflow import keras

### SQL連線(from GCP)

In [3]:
username = 'cfb101spade'
password = pwd
host_port = host_port
database = 'twfruits'

engine = create_engine("mysql+pymysql://{}:{}@{}/{}".format(username, password, host_port, database))
con = engine.connect()

In [4]:
fruit = "banana"
market = "台北二"
df= pd.read_sql(f"{fruit}_{market}", engine)
df

,日期,banana_平均價,banana_交易量,scarletbanana_平均價,guava_平均價,banana_前日平均價,banana_5日平均價
0,2011-01-01,31.4,9645.0,36.5,45.0,31.4,32.2
1,2011-01-02,32.0,12230.0,50.0,47.5,31.4,32.2
2,2011-01-03,32.2,12103.0,46.2,47.5,32.0,32.2
3,2011-01-04,32.4,11976.0,42.5,47.5,32.2,32.2
4,2011-01-05,33.2,7520.0,34.8,45.8,32.4,32.2
...,...,...,...,...,...,...,...
3878,2021-08-14,47.0,6695.0,29.3,38.6,45.1,48.0
3879,2021-08-15,48.4,6681.0,34.8,49.4,47.0,48.4
3880,2021-08-16,45.5,8187.0,34.9,48.8,48.4,47.4
3881,2021-08-17,42.6,9693.0,35.0,48.3,45.5,45.7


### 預測價格
- 從DB讀取資料 -> 讀取model -> 預測

In [18]:
from my_libs.my_model_trainer import *

In [20]:
fruits = {"banana": ["banana", "scarletbanana", "guava"],
          "guava":  ["guava", 'emperorguava', 'banana']}

markets = ['台北二', '台北一', '三重區', '台中市']

pred_output = []
actual_output = []

# date
today = datetime.now().strftime('%Y-%m-%d')
future_dates = list(pd.date_range(start=today, freq="d", periods=7))

# sql schema
df_for_sql = pd.DataFrame(columns=["水果","市場", "日期", "價格"])

# prediction
for Fruit, Type in fruits.items():
    for market in markets:
        
        product = Type[0]
        same = Type[1]
        sub = Type[2]
        
        # 從SQL讀取分析資料，轉換成矩陣
        df_set = pd.read_sql_query(f"""select `{product}_平均價`, `{product}_交易量`, `{same}_平均價`, 
                                       `{sub}_平均價`, `{product}_前日平均價`, `{product}_5日平均價` 
                                       from {product}_{market}
                                       order by `日期` desc
                                       limit 30""", con).values 
        
        # 標準化
        df_set_sc, sc_target = data_normalization(df_set)
        
        # reshape => 需符合input的shape
        df_set_sc = df_set_sc.reshape(-1, 30, 6)
        
        # 載入model
        file_name = f"C:/Users/Tibame/PycharmProjects/PythonETL/Fruit Price Prediction/models/{Fruit}_predict_adjusted/{market}.h5"
        model_loaded = keras.models.load_model(file_name)
        
        # 進行預測
        pred_result = model_loaded(df_set_sc)  # 使用model_loaded.predict(df_set_sc)會跳出tensorflow warning
        
        # 透過inverse_transform還原為原本數值
        output = sc_target.inverse_transform(pred_result).reshape(-1).round(2)
        
        for date, price in zip(future_dates, output):
            df_for_sql = df_for_sql.append({"水果":Fruit, "市場":market, "日期":date, "價格":price}, ignore_index=True)
        
        print(f"{Fruit}_{market}市場未來7日預測價: ", output)


banana_台北二市場未來7日預測價:  [51.13 50.54 51.72 50.26 50.43 51.42 50.14]
banana_台北一市場未來7日預測價:  [34.15 33.23 34.79 33.81 33.38 32.78 33.79]
banana_三重區市場未來7日預測價:  [33.23 31.99 31.23 33.74 33.09 31.95 33.16]
banana_台中市市場未來7日預測價:  [30.39 32.46 31.81 31.49 32.85 31.55 31.8 ]
guava_台北二市場未來7日預測價:  [38.72 38.17 38.68 37.53 38.47 38.75 38.45]
guava_台北一市場未來7日預測價:  [32.88 33.69 33.23 32.88 32.65 33.22 33.03]
guava_三重區市場未來7日預測價:  [26.3  26.31 26.46 26.48 26.28 26.21 26.58]
guava_台中市市場未來7日預測價:  [23.56 23.91 23.85 23.94 23.68 23.38 23.67]


In [17]:
df_for_sql.head()

,水果,市場,日期,價格
0,banana,台北二,2021-08-20,51.13
1,banana,台北二,2021-08-21,50.54
2,banana,台北二,2021-08-22,51.72
3,banana,台北二,2021-08-23,50.26
4,banana,台北二,2021-08-24,50.43


### 預測結果存回DB

In [15]:
df_for_sql["價格"] = df_for_sql["價格"].apply(lambda x:round(x,2))

In [16]:
# 預測結果寫入MySQL中
df_for_sql.to_sql(name='predicted price', 
                  con=con, if_exists='append',
                  index=False)